In [246]:
import folium
import pandas as pd
import json
from pytz import country_timezones
import requests
import numpy as np
from numpy import float32

In [247]:
import geopandas as gpd

assert "naturalearth_lowres" in gpd.datasets.available
datapath = gpd.datasets.get_path("naturalearth_lowres")
gdf = gpd.read_file(datapath)

In [248]:
hunger_data = pd.read_csv('prevalence-of-undernourishment.csv')

In [249]:
hunger_data.head()

,Entity,Code,Year,Prevalence of undernourishment (Percent of population)
0,Afghanistan,AFG,2001,47.799999
1,Afghanistan,AFG,2002,45.599998
2,Afghanistan,AFG,2003,40.599998
3,Afghanistan,AFG,2004,38.000000
4,Afghanistan,AFG,2005,36.099998


In [250]:
hunger_data.set_index('Year')

,Entity,Code,Prevalence of undernourishment (Percent of population)
Year,,,
2001,Afghanistan,AFG,47.799999
2002,Afghanistan,AFG,45.599998
2003,Afghanistan,AFG,40.599998
2004,Afghanistan,AFG,38.000000
2005,Afghanistan,AFG,36.099998
...,...,...,...
2015,Yemen,YEM,43.400002
2016,Yemen,YEM,46.099998
2017,Yemen,YEM,46.599998


In [251]:
n_periods = 19



datetime_index = pd.date_range("2001-1-1", periods=n_periods, freq="Y")
dt_index_epochs = datetime_index.astype(int) // 10 ** 9
dt_index = dt_index_epochs.astype("U10")

dt_index

Index(['1009756800', '1041292800', '1072828800', '1104451200', '1135987200',
       '1167523200', '1199059200', '1230681600', '1262217600', '1293753600',
       '1325289600', '1356912000', '1388448000', '1419984000', '1451520000',
       '1483142400', '1514678400', '1546214400', '1577750400'],
      dtype='object')

In [252]:
u_country = hunger_data.Entity.unique()

In [253]:
hunger_data[hunger_data['Code']== 'AFG']['Prevalence of undernourishment (Percent of population)'].to_numpy(dtype=float32)

array([47.8, 45.6, 40.6, 38. , 36.1, 33.3, 29.8, 26.5, 24.4, 23.7, 24.7,
       28.2, 26.3, 24.2, 21.5, 22.2, 23. , 23.4, 25.6], dtype=float32)

In [254]:
hunger_data.Year.unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [255]:

styledata = {}
feature_id = 0
for country in hunger_data.Code.unique():
    #print(country)
    if country == country:
        df = pd.DataFrame(
            {
                "name":  country,
                "color": hunger_data[hunger_data['Code']== country]['Prevalence of undernourishment (Percent of population)'].to_numpy(dtype=float32),
                "opacity": np.ones(19)*0.8,
            },
            #index=hunger_data.Year.unique()
            index=dt_index,
        )
        df.sample(1, replace=False).sort_index()
        styledata[feature_id] = df
        feature_id = feature_id + 1
print(styledata)

{0:            name      color  opacity
1009756800  AFG  47.799999      0.8
1041292800  AFG  45.599998      0.8
1072828800  AFG  40.599998      0.8
1104451200  AFG  38.000000      0.8
1135987200  AFG  36.099998      0.8
1167523200  AFG  33.299999      0.8
1199059200  AFG  29.799999      0.8
1230681600  AFG  26.500000      0.8
1262217600  AFG  24.400000      0.8
1293753600  AFG  23.700001      0.8
1325289600  AFG  24.700001      0.8
1356912000  AFG  28.200001      0.8
1388448000  AFG  26.299999      0.8
1419984000  AFG  24.200001      0.8
1451520000  AFG  21.500000      0.8
1483142400  AFG  22.200001      0.8
1514678400  AFG  23.000000      0.8
1546214400  AFG  23.400000      0.8
1577750400  AFG  25.600000      0.8, 1:            name  color  opacity
1009756800  ALB    4.9      0.8
1041292800  ALB    6.1      0.8
1072828800  ALB    7.5      0.8
1104451200  ALB    8.6      0.8
1135987200  ALB    8.9      0.8
1167523200  ALB    8.8      0.8
1199059200  ALB    8.3      0.8
1230681600  ALB 

In [256]:
styledata[0]

,name,color,opacity
1009756800,AFG,47.799999,0.8
1041292800,AFG,45.599998,0.8
1072828800,AFG,40.599998,0.8
1104451200,AFG,38.000000,0.8
1135987200,AFG,36.099998,0.8
1167523200,AFG,33.299999,0.8
1199059200,AFG,29.799999,0.8
1230681600,AFG,26.500000,0.8
1262217600,AFG,24.400000,0.8
1293753600,AFG,23.700001,0.8


In [257]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data["color"].max())
    min_color = min(max_color, data["color"].min())
    max_opacity = max(max_color, data["opacity"].max())
    max_opacity = min(max_color, data["opacity"].max())

In [258]:
from branca.colormap import linear

cmap = linear.YlOrRd_06.scale(0, 100)

def norm(x):
    return (x - x.min()) / (x.max() - x.min())


for country, data in styledata.items():
    print(data)
    data["color"] = data["color"].apply(cmap)
    #data["opacity"] = norm(data["opacity"])

           name      color  opacity
1009756800  AFG  47.799999      0.8
1041292800  AFG  45.599998      0.8
1072828800  AFG  40.599998      0.8
1104451200  AFG  38.000000      0.8
1135987200  AFG  36.099998      0.8
1167523200  AFG  33.299999      0.8
1199059200  AFG  29.799999      0.8
1230681600  AFG  26.500000      0.8
1262217600  AFG  24.400000      0.8
1293753600  AFG  23.700001      0.8
1325289600  AFG  24.700001      0.8
1356912000  AFG  28.200001      0.8
1388448000  AFG  26.299999      0.8
1419984000  AFG  24.200001      0.8
1451520000  AFG  21.500000      0.8
1483142400  AFG  22.200001      0.8
1514678400  AFG  23.000000      0.8
1546214400  AFG  23.400000      0.8
1577750400  AFG  25.600000      0.8
           name  color  opacity
1009756800  ALB    4.9      0.8
1041292800  ALB    6.1      0.8
1072828800  ALB    7.5      0.8
1104451200  ALB    8.6      0.8
1135987200  ALB    8.9      0.8
1167523200  ALB    8.8      0.8
1199059200  ALB    8.3      0.8
1230681600  ALB    7.3  

In [259]:
styledata

{0:            name      color  opacity
 1009756800  AFG  #fea446ff      0.8
 1041292800  AFG  #fea847ff      0.8
 1072828800  AFG  #feb14bff      0.8
 1104451200  AFG  #feb650ff      0.8
 1135987200  AFG  #feba54ff      0.8
 1167523200  AFG  #febf5aff      0.8
 1199059200  AFG  #fec661ff      0.8
 1230681600  AFG  #fecd68ff      0.8
 1262217600  AFG  #fed16dff      0.8
 1293753600  AFG  #fed26eff      0.8
 1325289600  AFG  #fed06cff      0.8
 1356912000  AFG  #fec965ff      0.8
 1388448000  AFG  #fecd69ff      0.8
 1419984000  AFG  #fed16dff      0.8
 1451520000  AFG  #fed673ff      0.8
 1483142400  AFG  #fed571ff      0.8
 1514678400  AFG  #fed370ff      0.8
 1546214400  AFG  #fed36fff      0.8
 1577750400  AFG  #fece6aff      0.8,
 1:            name      color  opacity
 1009756800  ALB  #fff6a3ff      0.8
 1041292800  ALB  #fff4a0ff      0.8
 1072828800  ALB  #fff19cff      0.8
 1104451200  ALB  #ffef98ff      0.8
 1135987200  ALB  #ffef97ff      0.8
 1167523200  ALB  #ffef98ff    

In [260]:
styledata.get(0).head()

,name,color,opacity
1009756800,AFG,#fea446ff,0.8
1041292800,AFG,#fea847ff,0.8
1072828800,AFG,#feb14bff,0.8
1104451200,AFG,#feb650ff,0.8
1135987200,AFG,#feba54ff,0.8


In [261]:
styledict = {
    country: data.to_dict(orient="index") for country, data in styledata.items()
}

In [270]:
pd.options.display.max_rows = 999
print(gdf.iso_a3)


0      FJI
1      TZA
2      ESH
3      CAN
4      USA
5      KAZ
6      UZB
7      PNG
8      IDN
9      ARG
10     CHL
11     COD
12     SOM
13     KEN
14     SDN
15     TCD
16     HTI
17     DOM
18     RUS
19     BHS
20     FLK
21     NOR
22     GRL
23     ATF
24     TLS
25     ZAF
26     LSO
27     MEX
28     URY
29     BRA
30     BOL
31     PER
32     COL
33     PAN
34     CRI
35     NIC
36     HND
37     SLV
38     GTM
39     BLZ
40     VEN
41     GUY
42     SUR
43     FRA
44     ECU
45     PRI
46     JAM
47     CUB
48     ZWE
49     BWA
50     NAM
51     SEN
52     MLI
53     MRT
54     BEN
55     NER
56     NGA
57     CMR
58     TGO
59     GHA
60     CIV
61     GIN
62     GNB
63     LBR
64     SLE
65     BFA
66     CAF
67     COG
68     GAB
69     GNQ
70     ZMB
71     MWI
72     MOZ
73     SWZ
74     AGO
75     BDI
76     ISR
77     LBN
78     MDG
79     PSE
80     GMB
81     TUN
82     DZA
83     JOR
84     ARE
85     QAT
86     KWT
87     IRQ
88     OMN
89     VUT
90     KHM

In [265]:
import folium
from folium.plugins import TimeSliderChoropleth


m = folium.Map([0, 0], zoom_start=2)
folium.TileLayer('stamenterrain').add_to(m)




g = TimeSliderChoropleth(
    gdf.to_json(),
    styledict=styledict,
).add_to(m)

m

In [263]:
from folium.plugins import TimeSliderChoropleth
world = folium.Map(location=[5.60, 0.19], zoom_start=2.5, control_scale=True)
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
world_countries = f'{url}/world-countries.json'
geo_json_data = json.loads(requests.get(world_countries).text)
folium.TileLayer('stamenterrain').add_to(world)
folium.TileLayer('stamen toner').add_to(world)
# Create a layer on the map with the prevelance of undernourishment as a choropleth layer
folium.Choropleth(
    geo_data=geo_json_data,
    name="choropleth",
    data=hunger_data,
    columns=["Code", "Prevalence of undernourishment (Percent of population)"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.7,
    legend_name="Prevalence of undernourishment (Percent of population)",
).add_to(world)
# This adds a layer control to the map so a user can select which layer(s) to display
folium.LayerControl().add_to(world)
world

